# Build a tile index csv and Mosaic Json for COGs

##### Path to your icesat2_boreal/lib dir - clone the icesat2_boreal GitHub repository
https://github.com/lauraduncanson/icesat2_boreal.git

In [1]:
ICESAT2_BOREAL_REPO_PATH = '/projects/Developer/icesat2_boreal' # /projects/Developer/icesat2_boreal/lib
ICESAT2_BOREAL_LIB_PATH = ICESAT2_BOREAL_REPO_PATH + '/lib'

In [2]:
!pip install -U -r $ICESAT2_BOREAL_REPO_PATH/dps/requirements_main.txt

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
import geopandas
import pandas as pd
import os
import json
import collections
import numpy as np
import sys
import s3fs
import matplotlib.pyplot as plt
sys.path.append(ICESAT2_BOREAL_LIB_PATH)
import maplib_folium
import ExtractUtils

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Build the tile index csv

In [4]:
DPS_DATA_TYPE = 'HLS' #'AGB'
DPS_ALG_NAME = 'do_HLS_stack_3-1-2_ubuntu'
DPS_RUN = 'spring2022'
OUT_MASTER_CSV_JSON_DIR = '/projects/my-public-bucket/DPS_tile_lists/'+DPS_RUN
OUT_MASTER_CSV_JSON_DIR = '/projects/my-public-bucket/DPS_tile_lists/HLS_test_redo/'+DPS_RUN
#DPS_DIR = 's3://maap-ops-workspace/lduncanson/dps_output/run_boreal_biomass_v4_ubuntu/master/2022/norway_mask1'

AGB_tindex_master_fn    = os.path.join(OUT_MASTER_CSV_JSON_DIR, f'{DPS_DATA_TYPE}_tindex_master.csv')
tile_matches_geojson_fn = os.path.join(OUT_MASTER_CSV_JSON_DIR, f'{DPS_DATA_TYPE}_tindex_master.json')
out_mosaic_json_fn      = os.path.join(OUT_MASTER_CSV_JSON_DIR, f'{DPS_DATA_TYPE}_tindex_master_mosaic.json')

In [5]:
#!python $ICESAT2_BOREAL_LIB_PATH/build_tindex_master.py --type ATL08_filt -m $DPS_RUN -o $OUT_MASTER_CSV_JSON_DIR

In [6]:
!python $ICESAT2_BOREAL_LIB_PATH/build_tindex_master.py --type $DPS_DATA_TYPE -alg_name $DPS_ALG_NAME -m $DPS_RUN -o $OUT_MASTER_CSV_JSON_DIR

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string

Building a list of tiles:
MAAP version:		master
Type:		HLS
Year:		2022
Month:		['spring2022']
Days:		1-31

Output dir:  /projects/my-public-bucket/DPS_tile_lists/HLS_test_redo/spring2022
                                             s3_path  ...                                file
0  s3://maap-ops-workspace/nathanmthomas/dps_outp...  ...     HLS_7_06-01_09-15_2019_2021.tif
1  s3://maap-ops-workspace/nathanmthomas/dps_outp...  ...   HLS_178_06-01_09-15_2019_2021.tif
2  s3://maap-ops-workspace/nathanmthomas/dps_outp...  ...   HLS_275_06-01_09-15_2019_2021.tif
3  s3://maap-ops-workspace/nathanmthomas/dps_outp...  ...  HLS_3848_06-01_09-15_2019_2021.tif
4  s3://maap-ops-workspace/nathanmthomas/dps_

In [7]:
tindex_master = pd.read_csv(AGB_tindex_master_fn)
tindex_master.head()
len(tindex_master)

4240

### Get all boreal tiles

In [8]:
boreal_tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg' 
boreal_tile_index = geopandas.read_file(boreal_tile_index_path)

boreal_tile_index["tile_num"] = boreal_tile_index["tile_num"].astype(int)

#fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,10))
#boreal_tile_index.plot(column='tile_num', linewidth=0.1, legend=True, ax=ax, cmap='Spectral')

DriverError: /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg: Transport endpoint is not connected

In [ ]:
AGB_TILES_NEEDED = ExtractUtils.GET_TILES_NEEDED(DPS_DATA_TYPE = DPS_DATA_TYPE, \
                                                tindex_master_fn = AGB_tindex_master_fn,\
                                                boreal_tile_index_path = boreal_tile_index_path)

In [ ]:
BAD_TILE_LIST = [3540,3634,3728,3823,3916,4004]

# For the tindex_master, convert it into vector tiles that show the tiles we have
cols_list = ['tile_num','s3_path','local_path']

# Select the rows we have results for
tile_index_matches = boreal_tile_index.merge(tindex_master[~tindex_master['tile_num'].isin(BAD_TILE_LIST)][cols_list], how='right', on='tile_num')
tile_index_matches = tile_index_matches[tile_index_matches['s3_path'].notna()]
tile_index_matches.head()

## Build a MosaicJSON

##### Build tile geojsons needed for mosaic jsons

In [ ]:
# Corrections were made to ensure GeoJSON *_tindex_master.json was set correctly to 4326
tile_matches_geojson_string = tile_index_matches.to_crs("EPSG:4326")

#Write copy to disk for debug 
tile_matches_geojson_string.to_file(tile_matches_geojson_fn, driver='GeoJSON')

boreal = geopandas.read_file('/projects/my-public-bucket/analyze_agb/input_zones/wwf_circumboreal_Dissolve.geojson')
ax = boreal.boundary.plot(color='black')
tile_matches_geojson_string.plot(ax=ax)
tile_matches_geojson_string = tile_matches_geojson_string.to_json()

# This is formatted nicely (printed)
tile_matches_geojson = json.loads(tile_matches_geojson_string)

In [ ]:

from typing import Dict
from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

def get_accessor(feature: Dict):
    """Return specific feature identifier."""
    return feature["properties"]["s3_path"]

#out_mosaic_json_fn = f's3://maap-ops-workspace/shared/{DPS_DATA_USER}/DPS_tile_lists/{DPS_DATA_TYPE}_tindex_master_mosaic.json' 
#out_mosaic_json_fn = f's3://maap-ops-workspace/shared/alexdevseed/DPS_tile_lists/{DPS_DATA_TYPE}_tindex_master_mosaic.json' 

print(f"Building {out_mosaic_json_fn}")
mosaicdata = MosaicJSON.from_features(tile_matches_geojson.get('features'), minzoom=6, maxzoom=18, accessor=get_accessor)

with MosaicBackend(out_mosaic_json_fn, mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)

### Build a mosaic json of a test local tile

In [ ]:
if False:
    TEST_TILE_NUM = 42 #3000

    # -- Adjust this manually
    tmp = boreal_tile_index[boreal_tile_index.tile_num == TEST_TILE_NUM]
    #tmp['s3_path'] = f's3://maap-ops-workspace/shared/nathanmthomas/test_hls/test_mc40_3yrs/HLS_{TEST_TILE_NUM}_06-01_09-15_2019_2021.tif'
    tmp['s3_path'] =  f's3://maap-ops-workspace/shared/lduncanson/testing/output/boreal_agb_202205041651688894_00{TEST_TILE_NUM}.tif'
    #out_mosaic_json_TEST_fn = f'/projects/my-public-bucket/test_hls/test_mc40_3yrs/HLS_tile_test_mosaic_tile_{TEST_TILE_NUM}.json'

    tile_matches_geojson_TEST_fn = f'/projects/my-public-bucket/TEST_tindex_{TEST_TILE_NUM}.json'
    out_mosaic_json_TEST_fn = f'/projects/my-public-bucket/TEST_tindex_mosaic_{TEST_TILE_NUM}.json'
    # --


    # Corrections were made to ensure GeoJSON *_tindex_master.json was set correctly to 4326
    tile_matches_geojson_string = tmp.to_crs("EPSG:4326")

    #Write copy to disk for debug 
    tile_matches_geojson_string.to_file(tile_matches_geojson_TEST_fn, driver='GeoJSON')

    tile_matches_geojson_string.plot()
    tile_matches_geojson_string = tile_matches_geojson_string.to_json()

    # This is formatted nicely (printed)
    tile_matches_geojson = json.loads(tile_matches_geojson_string)

    print(f"Building {out_mosaic_json_TEST_fn}")
    mosaicdata = MosaicJSON.from_features(tile_matches_geojson.get('features'), minzoom=6, maxzoom=18, accessor=get_accessor)

    with MosaicBackend(out_mosaic_json_TEST_fn, mosaic_def=mosaicdata) as mosaic:
        mosaic.write(overwrite=True)

    out_mosaic_json_TEST_fn